### Импорт библиотек

In [12]:
import random
import pandas as pd
from datetime import datetime, timedelta

### Множества входных данных

In [13]:
full_names = [
    "Семёнов Дмитрий Иванович",
    "Головин Сергей Вальерьевич",
    "Путилов Андрей Маркович",
    "Лапухов Алексей Дмитриевич",
    "Дружинин Георгий Михайлович"
    ]

status = [
    "Вход",
    "Выход",
    "Доступ запрещён"
]

### Формирование отчётного периода

In [14]:
def GenerateReportingPeriod():
    start_date = datetime(
        year=2006,
        month=4,
        day=1,
        hour=8,
        minute=0,
        second=0
    )

    stop_date = start_date.replace(month=7)
    date_range = (stop_date - start_date).days
    
    return [start_date + timedelta(days=day) 
            for day in range(date_range) 
            if (start_date + timedelta(days=day)).weekday() not in [5, 6]
    ]

### Генерация сырых данных

Кратко опишем генерацию времени входа и времени выхода. Пусть нормой считается начало дня в 9:00, а конец дня в 18:00, тогда время рабочего дня $t_{норм} = 9$.

Без псевдослучайного смещения имеем нормы входа и выхода 
$$
t_{входа} = 8 \\
t_{выхода} = 8 + t_{норм}
$$

Вводим псевдослучайное смещение, определяемое следующим образом:
$$
0 \leq t_{смещения} \leq 9059 \text{ (в секундах)}
$$

Таким образом, наименьшее смещение - 0 секунд, наибольшее смещение - 2 часа 30 минут и 59 секунд.
Получаем следующие нормы входа и выхода:
$$
t_{входа} = 8 + t_{смещения} \\
t_{выхода} = 8 + t_{норм} + t_{смещения}
$$

Соответственно, имеем следующие возможные значения в формате ЧЧ:ММ:СС:
$$
min(t_{входа}) = 08:00:00, \, max(t_{входа}) = 10:30:59 \\
min(t_{выхода}) = 17:00:00, \, max(t_{выхода}) = 19:30:59
$$

Опишем некоторые интервалы для точности их классификации. Пусть норма начала дня - $09:00:00$, норма окончания дня - $18:00:00$ и $\exists \, t_{входа}^{i}$ и $t_{выхода}^{i}$, тогда
$$
08:00:00 \leq t_{входа}^{i} \leq 09:00:00 \text{ - норма}, \\
09:00:00 < t_{входа}^{i} \leq 10:30:59 \text{ - опоздание}, \\
17:00:00 \leq t_{выхода}^{i} < 18:00:00 \text{ - ранний выход}, \\
18:00:00 \leq t_{выхода}^{i} \leq 19:30:59 \text{ - норма}. 
$$

Примечание: результирующая структура не содержит аномалий, описанных в пунктах тз после слов "следует учесть".

In [15]:
def GetProbability() -> bool:
    if random.randint(0, 99) < 4:
        return True

    return False

def GetTimeOffset(hours_max_offset=2, minutes_max_offset=30, seconds_max_offset=59) -> timedelta:
    return timedelta(
            hours=random.randint(0, hours_max_offset),
            minutes=random.randint(0, minutes_max_offset),
            seconds=random.randint(0, seconds_max_offset)
        )

def GenerateData(reporting_period : list) -> pd.DataFrame:
    acc = []

    for current_date in reporting_period:
        for name in full_names:
            acc.append({
                "full_name" : name,
                "event_dt" : current_date + GetTimeOffset(),
                "status" : status[0]
            })

            if GetProbability():
                working_hours = timedelta(hours=18)
            else:
                working_hours = timedelta(hours=9)

            acc.append({
                "full_name" : name,
                "event_dt" : current_date + working_hours + GetTimeOffset(),
                "status" : status[1]
            })

    return pd.DataFrame(acc)

## Аномализация данных

В данном разделе описанны функции, совершающие "иньекции" в полученный на предыдущем этапе датасет. Под "иньекцией" подразумевается вставка аномальных данных или замена сырых данных на аномальные. Для каждого типа аномалии реализована отдельная функция. 

### Ошибки системы
Рассматриваются случаи, описанные в тз, описываемые как внешняя причина для создания повторной записи с тем же статусом. В данных это отображается копией записи с временной разницей по сравнению с оригиналом в несколько секнуд. Будем считать, что физическая система турникетов допускает такую ситуацию в 5% случаев.

In [16]:
def GenerateErrors(data : pd.DataFrame) -> pd.DataFrame:
    errors_amount = round(len(data) * 0.01)
    samples = data.sample(errors_amount)
    copies = samples.copy()

    copies["event_dt"] += GetTimeOffset(hours_max_offset=0, minutes_max_offset=0, seconds_max_offset=32)

    return copies

### Покидание офиса не через турникет

Предполагаемая ситуация - покидание здания через запасный выход, минуя турникет. Исходя из описанных в тз ситуаций следует, что возвращение в здание происходит через турникет, т.е. существует запись о входе - непосредственных приход на работу, далее, опуская случаи, подразумевающие пару записей вход-выход, существует непарная запись о входе, как раз предполагающая описываемую в данном разделе ситуацию.

In [17]:
def GenerateEmergency(reporting_period : list) -> list:
    acc = []

    dates = random.sample(reporting_period, 4)

    for current_date in dates:
        emergency_date = current_date.replace(hour=15, minute=00, second=0)

        for name in full_names:
            acc.append({
                "full_name" : name,
                "event_dt" : emergency_date + GetTimeOffset(hours_max_offset=0, minutes_max_offset=10, seconds_max_offset=59),
                "status" : status[0]
            })

    return acc

### Перерывы на обед

In [18]:
def GenerateDinner(reporting_period : list) -> list:
    dinner_names = full_names[:2]
    acc = []

    for current_date in reporting_period:
        dinner_date = current_date.replace(hour=12, minute=45, second=0)

        for name in dinner_names:
            acc.append({
                "full_name" : name,
                "event_dt" : dinner_date + GetTimeOffset(hours_max_offset=0, minutes_max_offset=30, seconds_max_offset=59),
                "status" : status[1]
            })

            acc.append({
                "full_name" : name,
                "event_dt" : dinner_date + timedelta(minutes=30) + GetTimeOffset(hours_max_offset=0, minutes_max_offset=30, seconds_max_offset=59),
                "status" : status[0]
            })

    return acc

### "Доступ запрещён"
Для полноты раскрытия этого статуса нужно рассмотреть две ситуации:
1. Ожидаемый статус - "Вход"
2. Ожидаемый статус - "Выход"

В первом случае вход невозможен, предположим, что это считается нормой, например, забывчивый сотрудник попытался прийти в выходной за свой счёт на работу, тогда для отображения такого случая в данных необходимо заменить запись о входе этого сотрудника, заменить её статус на "Доступ запрещён", а другие записи, относящиеся к этим же дате и сотруднику, удалить.

Во втором случае такой статус расценивается как "Выход", тогда для отображения такого случая достаточно заменить статус записи "Выход" на "Доступ запрещён".

In [19]:
def FindRestrictedEnter(data : pd.DataFrame) -> tuple:
    restricted_amount = round(len(data) * 0.02)
    
    entries = data[data["status"] == status[0]]
    samples = entries.sample(restricted_amount)

    left = data.assign(_idx=data.index, date_key=data["event_dt"].dt.date)
    right = samples.assign(date_key=samples["event_dt"].dt.date)
    merged = left.merge(right[["full_name", "date_key"]], on=["full_name", "date_key"], how="inner")

    drop_idx = pd.Index(merged["_idx"]).difference(samples.index).to_list()
    update_idx = samples.index.to_list()

    return (drop_idx, update_idx)

def FindRestrictedExit(data : pd.DataFrame) -> list:
    restricted_amount = round(len(data) * 0.02)

    exits = data[data["status"] == status[1]]
    samples = exits.sample(restricted_amount)

    return samples.index.to_list()

### Выход ночью
На этапе генерации базового датасета были учтены ночные выходы, однако, для полноты представляемой картины попробуем воспроизвести ситуацию описанную в тз - выход, не попадающий в отчётный период. Для этого просто найдём последнюю запись о выходе и удалим её.

In [20]:
def FindLastExit(data : pd.DataFrame) -> int | str:
    exits = data[data["status"] == status[1]]
    latest_idx = exits["event_dt"].idxmax()

    return latest_idx

## "Сборка" датасета

In [21]:
reporting_period = GenerateReportingPeriod()
data = GenerateData(reporting_period)

print(f'Количество дней в отчётном периоде: {len(reporting_period)}')
print(f'Количество вхождений в базовый датасет: {len(data)}')

errors = GenerateErrors(data)
dinner = GenerateDinner(reporting_period)
emergency = GenerateEmergency(reporting_period)

drop_enter_idx, update_enter_idx = FindRestrictedEnter(data)
data.drop(drop_enter_idx, inplace=True)
data.loc[update_enter_idx, "status"] = status[2]

update_exit_idx = FindRestrictedExit(data)
data.loc[update_exit_idx, "status"] = status[2]

drop_latest_idx = FindLastExit(data)
data.drop(drop_latest_idx, inplace=True)

print(f'Количество вхождений в датасет, учитывающий случаи "Доступ запрещён" и "Выход ночью": {len(data)}')

data = pd.concat([data, errors, pd.DataFrame(dinner), pd.DataFrame(emergency)], ignore_index=True)

print(f'Количество вхождений в датасет, учитывающий все рассматриваемые случаи: {len(data)}')
print(data)

data.to_csv("data/source/entries.csv", index=False)

Количество дней в отчётном периоде: 65
Количество вхождений в базовый датасет: 650
Количество вхождений в датасет, учитывающий случаи "Доступ запрещён" и "Выход ночью": 634
Количество вхождений в датасет, учитывающий все рассматриваемые случаи: 920
                       full_name            event_dt status
0       Семёнов Дмитрий Иванович 2006-04-03 10:26:24   Вход
1       Семёнов Дмитрий Иванович 2006-04-03 19:27:08  Выход
2     Головин Сергей Вальерьевич 2006-04-03 10:07:37   Вход
3     Головин Сергей Вальерьевич 2006-04-03 18:28:17  Выход
4        Путилов Андрей Маркович 2006-04-03 08:15:27   Вход
..                           ...                 ...    ...
915     Семёнов Дмитрий Иванович 2006-05-03 15:07:44   Вход
916   Головин Сергей Вальерьевич 2006-05-03 15:10:21   Вход
917      Путилов Андрей Маркович 2006-05-03 15:08:57   Вход
918   Лапухов Алексей Дмитриевич 2006-05-03 15:00:04   Вход
919  Дружинин Георгий Михайлович 2006-05-03 15:05:05   Вход

[920 rows x 3 columns]
